In [1]:
using Luxor
using IntervalSets
using ForwardDiff
using Plots
gr()

function Bsp(p,k,i::Int64,t)::Float64
    if(p==0)
        return k[i]≤t<k[i+1]||(k[i]≠k[i+1]==k[end]==t)
    else
        B=0;
        if(k[i+p]-k[i]≠0)
            B+=Bsp(p-1,k,i,t)*(t-k[i])/(k[i+p]-k[i])
        end
        if(k[i+p+1]-k[i+1]≠0)
            B+=Bsp(p-1,k,i+1,t)*(k[i+p+1]-t)/(k[i+p+1]-k[i+1])
        end
        return B
    end
end

function cof(f,a,b)
    f(a),
    3*(f(2*a/3+b/3)-(8*f(a)+f(b))/27)-3*(f(a/3+2*b/3)-(f(a)+8*f(b))/27)/2,
    -3*(f(2*a/3+b/3)-(8*f(a)+f(b))/27)/2+3*(f(a/3+2*b/3)-(f(a)+8*f(b))/27),
    f(b)
end

function fastuniq(v)
  v1 = Vector{eltype(v)}()
  if length(v)>0
    laste = v[1]
    push!(v1,laste)
    for e in v
      if !(e ≈ laste)
        laste = e
        push!(v1,laste)
      end
    end
  end
  return v1
end

function Mesh(k,partn)
    K=fastuniq(k)
    K=[K[i]+(j/(partn))*(K[i+1]-K[i]) for i in 1:(length(K)-1), j in 0:(partn-1)]
    append!(sort(reshape(K,prod(size(K)))),k[end])
end

function BspCurve(N;name="NURBSmfd.svg",up=5,down=-5,right=5,left=-5,step=50)
    d,p,k,w,a=N

    P(t)=sum(w[i]Bsp(p,k,i,t)*a[i,:]/sum(w[j]Bsp(p,k,j,t) for j in 1:(length(k)-p-1)) for i in 1:(length(k)-p-1))
    
    Drawing(step*(right-left),step*(up-down),name)
    Luxor.origin(-step*left,step*up)
    background("white")
    sethue("red")
    K=fastuniq(k[1+p:end-p])
    move(step*Point([1,-1].*P(K[1])...))

    if((length(fastuniq(w))>1)||(p>3))
        K=Mesh(K,10)
    end
    N=length(K)
    for i in 1:(N-1)
        a₁,a₂,a₃,a₄=cof(t->P(t),K[i],K[i+1])
        curve(step*Point([1,-1].*a₂...),step*Point([1,-1].*a₃...),step*Point([1,-1].*a₄...))
    end
    strokepath()
    
    sethue("black")
    setline(1)
    Cp=[step*Point([1,-1].*a[i,:]...) for i in 1:size(a)[1]]
    map(p->circle(p,2,:fill), Cp)
    poly(Cp, :stroke)
    
    finish()
end

function href(N,k₊)
    (d,p,k,w,a)=N
    n=length(k)-p-1
    pᵣ=p
    kᵣ=sort(vcat(k,k₊))
    nᵣ=length(kᵣ)-pᵣ-1
    κ=[((nᵣ+1-i)/(nᵣ+1))*kᵣ[i]+(i/(nᵣ+1))*kᵣ[i+pᵣ+1] for i in 1:nᵣ]
    C=[Bsp(pᵣ,kᵣ,i,κ[j]) for j in 1:nᵣ, i in 1:nᵣ]
    D=C\[Bsp(p,k,j,κ[i]) for i in 1:nᵣ, j in 1:n]
    wᵣ=D*w
    aᵣ=[sum(w[i]*D[l,i]*a[i,j] for i in 1:n) for l in 1:nᵣ, j in 1:2]./wᵣ
    Nᵣ=(d,pᵣ,kᵣ,wᵣ,aᵣ)
end

function pref(N,p₊)
    (d,p,k,w,a)=N
    n=length(k)-p-1
    pᵣ=p+p₊
    k₊=repeat(fastuniq(k),inner=p₊)
    kᵣ=sort(vcat(k,k₊))
    nᵣ=length(kᵣ)-pᵣ-1
    κ=[((nᵣ+1-i)/(nᵣ+1))*kᵣ[i]+(i/(nᵣ+1))*kᵣ[i+pᵣ+1] for i in 1:nᵣ]
    C=[Bsp(pᵣ,kᵣ,i,κ[j]) for j in 1:nᵣ, i in 1:nᵣ]
    D=C\[Bsp(p,k,j,κ[i]) for i in 1:nᵣ, j in 1:n]
    wᵣ=D*w
    aᵣ=[sum(w[i]*D[l,i]*a[i,j] for i in 1:n) for l in 1:nᵣ, j in 1:2]./wᵣ
    Nᵣ=(d,pᵣ,kᵣ,wᵣ,aᵣ)
end

function SVGb(p,k;name="BCA.svg",up=5,down=-5,right=5,left=-5,step=50)
    N=length(k)
    
    Drawing(step*(right-left),step*(up-down),name)
    Luxor.origin(-step*left,step*up)
    background("white")
    sethue("blue")
    move(step*Point([1,-1].*p(k[1])...))
    for i in 1:(N-1)
        a₁,a₂,a₃,a₄=cof(t->p(t),k[i],k[i+1])
        curve(step*Point([1,-1].*a₂...),step*Point([1,-1].*a₃...),step*Point([1,-1].*a₄...))
    end
    strokepath()
    finish()
end

SVGb (generic function with 1 method)

In [7]:
d=1
p=3
k=[0,1,2.2,4,5,8,10.5,12,13,15]
k=[-5,-2,-1,0,5,8,10,11,12,15]
w=[1,1,4,1,1/3,1,1,1]
w=[1 for i in 1:(length(k)-p-1)]
a=(2*rand(length(k)-p-1,2)-1)*5
a=reshape([-3,-0.1,3,3.8,2,-2.5,
        -4,-3,-4.3,-1,3,2],6,2)+a/10
N=(d,p,k,w,a)
BspCurve(N,name="A.svg")
# k₊=[2.5];
# BspCurve(href(N,k₊),name="1dimh.svg")
# p₊=2
# BspCurve(pref(N,p₊),name="1dimp.svg")

SVGb((t->sum(Bsp(p,k,i,t)*a[i,:] for i in 1:(length(k)-p-1))),k,name="A-curve.svg")
for i in 1:(length(k)-p-1)
    SVGb(t->[t/5,Bsp(p,k,i,t)],k,name="A-Bsp"*string(i)*".svg",up=2,down=-1,right=maximum(k)/5+1,left=minimum(k)-1)
end
SVGb(t->[t/5,sum(Bsp(p,k,i,t) for i in 1:(length(k)-p-1))],k,name="Asum.svg",up=2,down=-1,right=maximum(k)/5+1,left=minimum(k)-1)

true

In [3]:
d=1
p=3
k=[0,0,0,0,5,8,10,10,10,10]
w=[1,1,4,1,1/3,1,1,1]
w=[1 for i in 1:(length(k)-p-1)]
a=(2*rand(length(k)-p-1,2)-1)*5
a=reshape([-3,-0.1,3,3.8,2,-2.5,
        -4,-3,-4.3,-1,3,2],6,2)+a/10
N=(d,p,k,w,a)
BspCurve(N,name="B.svg")
# k₊=[2.5];
# BspCurve(href(N,k₊),name="1dimh.svg")
# p₊=2
# BspCurve(pref(N,p₊),name="1dimp.svg")

for i in 1:(length(k)-p-1)
    SVGb(t->[t/5,Bsp(p,k,i,t)],k,name="B-Bsp"*string(i)*".svg",up=2,down=-1,right=maximum(k)/5+1,left=minimum(k)-1)
end
SVGb(t->[t/5,sum(Bsp(p,k,i,t) for i in 1:(length(k)-p-1))],k,name="Bsum.svg",up=2,down=-1,right=maximum(k)/5+1,left=minimum(k)-1)

true

In [15]:
d=1
p=3
k=[0,1,3,5,6,7,10,11,13,15,16,17,20]
w=[1 for i in 1:(length(k)-p-1)]
# a=(2*rand(length(k)-p-1,2)-1)*5
a=reshape([-3,-0.1,3,3.8,2,-2.5,-3,-0.1,3,
        -4,-3,-4.3,-1,3,2,-4,-3,-4.3],9,2)
N=(d,p,k,w,a)
BspCurve(N,name="C.svg")
# k₊=[2.5];
# BspCurve(href(N,k₊),name="1dimh.svg")
# p₊=2
# BspCurve(pref(N,p₊),name="1dimp.svg")

for i in 1:(length(k)-p-1)
    SVGb(t->[t/5,Bsp(p,k,i,t)],k,name="C-Bsp"*string(i)*".svg",up=2,down=-1,right=maximum(k)/5+1,left=minimum(k)-1)
end
SVGb(t->[t/5,sum(Bsp(p,k,i,t) for i in 1:(length(k)-p-1))],k,name="Csum.svg",up=2,down=-1,right=maximum(k)/5+1,left=minimum(k)-1)

true

In [14]:
length(k)-p-1

9

SVGb (generic function with 2 methods)

In [80]:
for i in 1:(length(k)-p-1)
    SVGb(t->[t/5,Bsp(p,k,i,t)],k,name="Bsp"*string(i)*".svg",up=2,down=-1,right=maximum(k)/5+1,left=minimum(k)-1)
end

In [85]:
(t->sum(Bsp(p,k,i,t)*a[i,:] for i in 1:(length(k)-p-1)))(0)

2-element Array{Float64,1}:
 0.0
 0.0

true

true